# Post Result Analysis

In [1]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

**Original Paper's last paragraph of page 8 and the first paragraph of page 9** [here](https://arxiv.org/pdf/1701.05927.pdf#8).<br><br>
One of the unique properties of high energy particle physics is that we have a library of useful jet observables that are physically motivated functions $f : \mathbb{R}^{25×25} → \mathbb{R}$ whose features are qualitatively (and in some cases, quantitatively) well understood. We can use the distributions of these observables to assess the abilty of the GAN to mimic Pythia. Three such features of a jet image ***$I$*** are the mass ***$m$***, transverse momentum ***$p_{T}$***, and n-subjettiness ***$\tau_{21}$***:

$$p^2_T(I) = \sum_{i}(I_icos(\phi_i))^2 + \sum_{i}(I_isin(\phi_i))^2$$
$$m^2(I) = \sum_{i}(I_i)^2 - p^2_T(I) - \sum_{i}(I_isinh(\eta_i))^2$$
$$\tau_n(I) \propto \sum_{i}(I_i)min_i(\sqrt{(\eta_i-\eta_a)^2-(\phi_i-\phi_a)^2})$$
$$\tau_{21}(I) = \frac{\tau_2(I)}{\tau_1(I)}$$

where ***$I_i$***, ***$\eta_i$***, and ***$\phi_i$*** are the pixel intensity, pseudorapidity, and azimuthal angle, respectively. The sums run over the entire image. The quantities $\eta_a$ and $\phi_a$ are axis values determined with the one-pass $k_t$ axis selection using the winner-take-all combination scheme.

### Discrete Mass

Calculation of the Jet Mass from a Pixelated Jet Image

### Discrete Momentum
Calculation of the Jet Momentum from a Pixelated Jet Image

### N-subjettiness

Calculation of N-subjettiness which is a jet shape designed to identify boosted hadronic objects such as top quarks.[here](https://arxiv.org/abs/1108.2701)